# Connection to LLM

Author: Pavel Agurov, pavel_agurov@epam.com

In this notebook we will see how to connect to LLM and run our first very simple query. "Query" is defined in model input and called "prompt". In fact, it's just a string.

We will see how to connect to Open AI, Azure and EPAM DIAL.

Please note: openAI is not secured. Do not send client's data, sensetive information to OpenAI.

In [ ]:
%pip install openai > /dev/null
%pip install tiktoken > /dev/null
%pip install langchain > /dev/null
%pip install langchain_openai > /dev/null
%pip install langchain_core > /dev/null
%pip install langchain_community > /dev/null
%pip install langchain_text_splitters > /dev/null

# EPAM DIAL connection

EPAM default connection is EPAM DIAL (https://epam-rail.com/). It's recommented way of connection.
EPAM DIAL has Azure connection. 

To request DIAL KEY please open https://chat.lab.epam.com/ and request your key on th bottom of the page. Please note, that it works only under EPAM VPN.

In [9]:
import os
from langchain_openai import AzureChatOpenAI
epam_dial = AzureChatOpenAI(
        api_key         = os.environ['OPENAI_API_KEY'],
        api_version     = "2023-07-01-preview",
        azure_endpoint  = "https://ai-proxy.lab.epam.com",
        model           = "gpt-4o-mini-2024-07-18",
        temperature     = 0.0
    )

epam_dial.invoke("Hello")

KeyboardInterrupt: 

## Open AI connection (template)

For Open AI we need only OPENAI_API_KEY. You can get it after registration on https://platform.openai.com/

There 2 ways to provide your key into LLM:
- set environment variable OPENAI_API_KEY (recommended)
- put your key into constructor

You can assign environment variable OPENAI_API_KEY from OS or read it from .env file.
See details here https://platform.openai.com/docs/quickstart?context=python

*Important!* Code below will NOT work without your api  key.

In [8]:
from langchain_openai import ChatOpenAI

llm_openAI = ChatOpenAI(
    model= "gpt-4o-mini-2024-07-18", 
    temperature = 0,
    # api_key="...",  # if you prefer to pass api key in directly instead of using env variables
)

llm_openAI.invoke('Hello!')      


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: dial-qgc********************jayu. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

# Azure connection (template)

You can also use Azure GPT. In this case we need a bit more parameters:
- AZURE_OPENAI_API_KEY as environment variable or passed into constructor
- API version
- Azure end point
- model name
- deployment name

*Important!* Code below will NOT work without your Azure endpoint and your key.

In [6]:
from langchain_openai import AzureChatOpenAI

your_azure_api_key = ""
azure_endpoint  = "https://???.openai.azure.com/"

llm_azure = AzureChatOpenAI(
        api_key         = your_azure_api_key,
        api_version     = "2023-07-01-preview",
        azure_endpoint  = azure_endpoint,
        model           = "gpt-4o-mini-2024-07-18",
        temperature     = 0.0
    )

llm_azure.invoke("Hello")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-35-turbo-16k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-76b90b4e-9097-4ad9-a32e-315813db1981-0')

# Output parser

It's not good idea to parse LLM answer, so we can add pre-defined parser. We will call it "chain".

In [6]:
from langchain_core.output_parsers import StrOutputParser

chain = llm_openAI | StrOutputParser()
chain.invoke('Hello!')

'Hello! How can I assist you today?'

Now we have only anwer. It's easy.

# Prompt

Let's add a prompt with parameters into our chain.

In [7]:
import textwrap
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# create prompt from template with parameters
prompt = ChatPromptTemplate.from_template("Tell me short shory about {subject}")

# create chain
chain = prompt | llm_openAI | StrOutputParser()

# now we will provide parameters as dict
story_text = chain.invoke({"subject" : "cat"})

# print result
print(f'{textwrap.fill(story_text, 80)}\n')

Once there was a curious cat named Whiskers who loved to explore the
neighborhood. One day, he stumbled upon a hidden garden filled with colorful
flowers and fluttering butterflies. Mesmerized by the beauty of the garden,
Whiskers spent hours chasing butterflies and rolling around in the soft grass.
From that day on, Whiskers would visit the garden every afternoon, finding peace
and joy in the simple pleasures of nature. The garden became his secret
sanctuary, a place where he could be free and happy. And so, Whiskers lived his
days content and carefree, always looking forward to his next adventure in the
magical garden.



# Used tokens

With the help of special callback we can find how many tokens were used during LLM call. You can find more information about tokens in notebooks 2.

In [8]:
import textwrap
from langchain_community.callbacks import get_openai_callback

with get_openai_callback() as cb:
    story_text = chain.invoke({"subject" : "dog"})
    
# print result
print(f"Token used: {cb.total_tokens}")
print(f'{textwrap.fill(story_text, 80)}\n')

Token used: 225
Once there was a stray dog named Max who roamed the streets in search of food
and shelter. Despite his rough appearance, Max had a kind heart and a gentle
demeanor. One day, a young girl named Lily found Max scavenging for scraps near
her house. She took pity on the poor dog and decided to bring him home with her.
At first, Max was hesitant and scared of his new surroundings, but Lily showed
him love and care, slowly earning his trust. Over time, Max became a loyal
companion to Lily, always by her side and ready to protect her from any danger.
As the days passed, Max's playful nature and unwavering loyalty brought joy and
happiness to Lily's life. They formed an unbreakable bond that transcended
words, a bond that only a dog and their human can share.  From that day on, Max
found a forever home with Lily, and together they lived happily ever after,
proving that sometimes all it takes is a little love and kindness to change a
stray dog's life for the better.

